In [1]:
from langchain_huggingface import ChatHuggingFace #to use huggingface llm class directly
from langchain_huggingface import HuggingFaceEndpoint #to run models serverless manner
from langchain_huggingface import HuggingFacePipeline #to run models locally
from langchain_huggingface import HuggingFaceEmbeddings #to run embedding model through dedicated endpoints
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings #to run embedding models serverless manner
from langchain_huggingface import HuggingFaceEmbeddings #to run embedding models locally
from langchain_community.embeddings import HuggingFaceInstructEmbeddings #to run embedding models locally
from langchain_community.embeddings import HuggingFaceBgeEmbeddings #to run bge embedding model

In [48]:
from huggingface_hub import login
login() # You will be prompted for your HF key, which will then be saved locally

In [20]:

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=10,
    do_sample=True,
    repetition_penalty=1.1,
)

In [19]:
res = llm.invoke("Who is the PM of India?")

StopIteration: 

In [23]:
chat = ChatHuggingFace(llm=llm, verbose=True)

In [29]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]

chat.invoke(messages)

ValueError: Model microsoft/phi-2 is not supported for task conversational and provider featherless-ai. Supported task: text-generation.

In [30]:
from langchain_huggingface import HuggingFacePipeline
import torch # Still needed for device settings if not using default CPU

llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/Phi-3-mini-4k-instruct", #"mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    device_map="cpu", # Automatically use GPU if available
    # torch_dtype=torch.bfloat16, # For memory efficiency
    pipeline_kwargs={
        "max_new_tokens": 10,
        "do_sample": True,
        "temperature": 0.1,
        "top_p": 0.95,
        "repetition_penalty": 1.1,
        # For Phi-3: "trust_remote_code": True, # This might go into model_kwargs if not directly supported by pipeline_kwargs
    },
    # You can also pass model_kwargs for arguments directly to model.from_pretrained
    # model_kwargs={"trust_remote_code": True},
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [9]:
llm.invoke("Who is the PM of India?")

'Who is the PM of India?\nOptions:\n(A) Narendra Modi (B) Rahul Gandhi (C) Amit Shah (D) Sonia Gandhi. The correct answer is (A) Narendra Modi, who has been serving as Prime Minister since May 26, 2014. He was re-elected for a second term in April 2019 and continues to hold office at present.'

In [28]:
chat = ChatHuggingFace(llm=llm, verbose=True)

messages = "<system>You are a helpful translator. Translate the user sentence to French.\n <human>I love programming."
# messages = [
#     ("system", "You are a helpful translator. Translate the user sentence to French."),
#     ("human", "I love programming."),
# ]

chat.invoke(messages)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [22]:
#alternativ we can make the hf pipeline and pass it through HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=10,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1,
)
llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\ritap\anaconda3\envs\data_science\Lib\site-packages\accelerate\utils\modeling.py:1582: UserWarning: Current model requires 64 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cpu


In [23]:
llm.invoke("Who is the PM of India?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Who is the PM of India?\nAnswer: Narendra Modi.\n\nExercise'

In [27]:
# tokenizer.chat_template = model_id
chat = ChatHuggingFace(llm=llm)

messages = "<system>You are a helpful translator. Translate the user sentence to French.\n <human>I love programming."
# messages = [
#     ("system", "You are a helpful translator. Translate the user sentence to French."),
#     ("human", "I love programming."),
# ]
chat.invoke(messages)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating